# FIT Library Showcase

This notebook demonstrates the features of the FIT library - a machine learning framework built with the help of NumPy.

The FIT library that I've created provides tensor operations with automatic differentiation, neural network components with inclusion of linear layers, activations, and normalization, attention mechanisms and transformers, multiple optimizers such as SGD, Adam, SAM, and Lion, loss functions for regression and classification, data pipeline utilities with built-in datasets, training monitoring and visualization, model persistence for saving and loading, and APIs for quick experimentation.

I tried to make this library lightweight and it only requires NumPy. It offers a PyTorch-like familiar API, and has an extensible clean architecture for adding components. 

# Quick Start

## Installation

In [1]:
# !pip install git+https://github.com/Klus3kk/fit.git

## Tensor Creation

In [2]:
import numpy as np
from fit.core.tensor import Tensor

# Create tensors from various data types
a = Tensor([1.0, 2.0, 3.0], requires_grad=True)
b = Tensor([[1, 2], [3, 4]], requires_grad=True)
c = Tensor(np.random.randn(3, 3), requires_grad=True)

print(f"Vector: {a}")
print(f"Matrix: {b}")
print(f"Random tensor: {c}")

Vector: Tensor([1. 2. 3.], requires_grad=True)
Matrix: Tensor([[1. 2.]
 [3. 4.]], requires_grad=True)
Random tensor: Tensor([[-1.08957361 -1.14300669 -1.06784566]
 [ 0.33019738 -0.22200625 -1.82231365]
 [ 0.33581401  1.90352181 -0.31104841]], requires_grad=True)


## Operations

In [3]:
# Basic arithmetic
x = Tensor([1.0, 2.0, 3.0], requires_grad=True)
y = Tensor([4.0, 5.0, 6.0], requires_grad=True)

# Addition and multiplication
z = x + y
w = x * y

print(f"x + y = {z.data}")
print(f"x * y = {w.data}")

# Matrix operations
A = Tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
B = Tensor([[2.0, 1.0], [4.0, 3.0]], requires_grad=True)
C = A @ B  # Matrix multiplication

print(f"Matrix multiplication result:\n{C.data}")

x + y = [5. 7. 9.]
x * y = [ 4. 10. 18.]
Matrix multiplication result:
[[10.  7.]
 [22. 15.]]


# Tensor Operations & Autograd

## Automatic DIfferentiation

In [4]:
from fit.core.tensor import Tensor

# Function: f(x) = x² + 2x + 1
x = Tensor([2.0], requires_grad=True)

# Forward pass
y = x * x + 2 * x + 1
print(f"f({x.data[0]}) = {y.data[0]}")

# Backward pass - compute gradient
y.backward()
print(f"f'({x.data[0]}) = {x.grad[0]}")
print(f"Expected derivative: {2 * x.data[0] + 2}")

f(2.0) = 9.0
f'(2.0) = 6.0
Expected derivative: 6.0


## Multi-variable functions

In [5]:
# Function: f(a, b) = sum(a * b)
a = Tensor([1.0, 2.0], requires_grad=True)
b = Tensor([3.0, 4.0], requires_grad=True)

# Forward pass
c = a * b
loss = c.sum()

# Backward pass
loss.backward()

print(f"Gradient of a = {a.grad}")
print(f"Gradient of b = {b.grad}")

Gradient of a = [3. 4.]
Gradient of b = [1. 2.]


## Chain Rule

In [6]:
# Complex function composition
x = Tensor([1.0], requires_grad=True)
y = x * 2
z = y + 3
w = z * z

w.backward()
print(f"Chain rule gradient: {x.grad[0]}")

Chain rule gradient: 20.0


# Neural Network Components

## Linear Layers

In [7]:
from fit.nn.modules.linear import Linear
from fit.core.tensor import Tensor

# Create a linear layer
layer = Linear(3, 2)  # 3 inputs -> 2 outputs

# Forward pass
x = Tensor([[1.0, 2.0, 3.0]], requires_grad=True)
output = layer(x)

print(f"Input shape: {x.data.shape}")
print(f"Output shape: {output.data.shape}")
print(f"Layer weights shape: {layer.weight.data.shape}")
print(f"Layer bias shape: {layer.bias.data.shape}")

Input shape: (1, 3)
Output shape: (1, 2)
Layer weights shape: (2, 3)
Layer bias shape: (2,)


## Activation Functions

In [8]:
from fit.nn.modules.activation import ReLU, Softmax, GELU

# Test different activations
x = Tensor([[-2.0, -1.0, 0.0, 1.0, 2.0]], requires_grad=True)

relu = ReLU()
softmax = Softmax()
gelu = GELU()

print(f"Input: {x.data}")
print(f"ReLU: {relu(x).data}")
print(f"Softmax: {softmax(x).data}")
print(f"GELU: {gelu(x).data}")

Input: [[-2. -1.  0.  1.  2.]]
ReLU: [[-0. -0.  0.  1.  2.]]
Softmax: [[0.01165623 0.03168492 0.08612854 0.23412166 0.63640865]]
GELU: [[-0.04540231 -0.15880801  0.          0.84119199  1.95459769]]


## Building Models

In [9]:
from fit.nn.modules.container import Sequential
from fit.nn.modules.linear import Linear
from fit.nn.modules.activation import ReLU, Softmax

# Create a neural network
model = Sequential(
    Linear(4, 8),  # Input layer
    ReLU(),  # Activation
    Linear(8, 6),  # Hidden layer
    ReLU(),  # Activation
    Linear(6, 3),  # Output layer
    Softmax(),  # Final activation
)

# Test the model
x = Tensor([[1.0, 2.0, 3.0, 4.0]], requires_grad=True)
output = model(x)

print(f"Model output: {output.data}")
print(f"Output sum (should be ~1.0): {output.data.sum()}")
print(f"Number of parameters: {len(model.parameters())}")

Model output: [[0.70130264 0.12593192 0.17276544]]
Output sum (should be ~1.0): 1.0
Number of parameters: 6


## Normalization Layers

In [10]:
from fit.nn.modules.normalization import BatchNorm, LayerNorm

# Batch normalization
batch_norm = BatchNorm(4)
x = Tensor([[1.0, 2.0, 3.0, 4.0], [2.0, 3.0, 4.0, 5.0]], requires_grad=True)
normalized = batch_norm(x)

print(f"Original: {x.data}")
print(f"Batch normalized: {normalized.data}")

# Layer normalization
layer_norm = LayerNorm(4)
layer_normalized = layer_norm(x)
print(f"Layer normalized: {layer_normalized.data}")

Original: [[1. 2. 3. 4.]
 [2. 3. 4. 5.]]
Batch normalized: [[-0.99998 -0.99998 -0.99998 -0.99998]
 [ 0.99998  0.99998  0.99998  0.99998]]
Layer normalized: [[-1.34163542 -0.44721181  0.44721181  1.34163542]
 [-1.34163542 -0.44721181  0.44721181  1.34163542]]


# Optimizers & Training

## SGD Optimizer

In [11]:
from fit.optim.sgd import SGD
from fit.nn.modules.linear import Linear

# Create model and optimizer
model = Linear(2, 1)
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

# Simulate training step
x = Tensor([[1.0, 2.0]], requires_grad=True)
target = Tensor([[3.0]])

# Forward pass
output = model(x)
loss = ((output - target) ** 2).mean()

# Backward pass
loss.backward()

# Optimizer step
optimizer.step()
optimizer.zero_grad()

print(f"Loss: {loss.data}")
print("Training step completed!")

Loss: 4.835619504464079
Training step completed!


## Adam Optimizer

In [12]:
from fit.optim.adam import Adam

# Create model and Adam optimizer
model = Sequential(Linear(2, 4), ReLU(), Linear(4, 1))

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

# Training loop simulation
for step in range(5):
    # Sample data
    x = Tensor([[np.random.randn(), np.random.randn()]], requires_grad=True)
    target = Tensor([[np.random.randn()]])

    # Forward pass
    output = model(x)
    loss = ((output - target) ** 2).mean()

    # Backward pass
    loss.backward()

    # Optimizer step
    optimizer.step()
    optimizer.zero_grad()

    # Fix: Handle both scalar and array loss values
    if loss.data.ndim == 0:
        # Scalar value
        loss_value = float(loss.data)
    else:
        # Array value
        loss_value = float(loss.data[0])

    print(f"Step {step+1}, Loss: {loss_value:.4f}")

Step 1, Loss: 0.8734
Step 2, Loss: 0.4638
Step 3, Loss: 0.8612
Step 4, Loss: 1.1857
Step 5, Loss: 0.6425


# Data Pipeline

## Custom dataset

In [13]:
from fit.data.dataset import Dataset
from fit.data.dataloader import DataLoader

# Create custom dataset
X = np.random.randn(100, 4)
y = np.random.randint(0, 3, 100)

dataset = Dataset(X, y)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

print(f"Dataset size: {len(dataset)}")
print(f"Number of batches: {len(dataloader)}")

# Iterate through batches
for i, (batch_x, batch_y) in enumerate(dataloader):
    print(f"Batch {i+1}: X shape {batch_x.data.shape}, y shape {batch_y.data.shape}")
    if i >= 2:  # Show first 3 batches
        break

Dataset size: 100
Number of batches: 7
Batch 1: X shape (16, 4), y shape (16,)
Batch 2: X shape (16, 4), y shape (16,)
Batch 3: X shape (16, 4), y shape (16,)


## Dataset Loaders

In [14]:
from fit.simple.data import load_dataset

# Load XOR dataset
xor_data = load_dataset("xor", batch_size=4)
print("XOR Dataset loaded:")
print(f"Train loader batches: {len(xor_data['train'])}")

# Load Iris dataset
iris_data = load_dataset("iris", batch_size=32, validation_split=0.2)
print("Iris Dataset loaded:")
print(f"Train batches: {len(iris_data['train'])}")
print(f"Validation batches: {len(iris_data['val'])}")

# Sample from XOR dataset
for x, y in xor_data["train"]:
    print(f"XOR - X: {x.data}, y: {y.data}")
    break

Generating XOR dataset...
XOR Dataset loaded:
Train loader batches: 250
Loading Iris dataset...
Iris Dataset loaded:
Train batches: 4
Validation batches: 1
XOR - X: [[0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]], y: [0. 1. 0. 1.]


## Feature Selection

In [15]:
from fit.data.feature_selection import SelectKBest, f_classif

# Generate sample data
X = np.random.randn(100, 10)
y = np.random.randint(0, 2, 100)

# Feature selection
selector = SelectKBest(score_func=f_classif, k=5)
X_selected = selector.fit_transform(X, y)

print(f"Original features: {X.shape[1]}")
print(f"Selected features: {X_selected.shape[1]}")
print(f"Selected feature indices: {selector.get_support()}")

Original features: 10
Selected features: 5
Selected feature indices: [ True False  True False False  True  True False False  True]


# Loss Functions

## Regression Loss

In [16]:
from fit.loss.regression import MSELoss

mse = MSELoss()

# Sample predictions and targets
predictions = Tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
targets = Tensor([[1.2, 1.8], [2.9, 4.1]])

loss = mse(predictions, targets)
print(f"MSE Loss: {loss.data}")

# Compute gradients
loss.backward()
print(f"Gradients: {predictions.grad}")

MSE Loss: 0.024999999999999977
Gradients: [[-0.1   0.1 ]
 [ 0.05 -0.05]]


## Classification Loss

In [17]:
from fit.loss.classification import CrossEntropyLoss

ce_loss = CrossEntropyLoss()

# Logits and class labels
logits = Tensor([[2.0, 1.0, 0.1], [0.5, 2.0, 0.8]], requires_grad=True)
targets = Tensor([0, 1])  # Class indices

loss = ce_loss(logits, targets)
print(f"CrossEntropy Loss: {loss.data}")

# Compute gradients
loss.backward()
print(f"Logits gradients: {logits.grad}")

CrossEntropy Loss: 0.419290646726997
Logits gradients: [[-0.17049943  0.12121649  0.04928295]
 [ 0.07318986 -0.17198583  0.09879597]]


# Attention & Transformers

## Attention Mechanism

In [18]:
from fit.nn.modules.attention import MultiHeadAttention

# Create multi-head attention
attention = MultiHeadAttention(d_model=64, num_heads=8)

# Sample input (batch_size=2, seq_len=10, d_model=64)
x = Tensor(np.random.randn(2, 10, 64), requires_grad=True)

# Self-attention
output = attention(x, x, x)
print(f"Attention input shape: {x.data.shape}")
print(f"Attention output shape: {output.data.shape}")

Attention input shape: (2, 10, 64)
Attention output shape: (2, 10, 64)


## Transformer Block

In [19]:
from fit.nn.modules.transformer import TransformerEncoderBlock

# Create transformer encoder block
transformer_block = TransformerEncoderBlock(
    d_model=64, num_heads=8, d_ff=256, dropout=0.1
)

# Forward pass
x = Tensor(np.random.randn(2, 10, 64), requires_grad=True)
output = transformer_block(x)

print(f"Transformer input shape: {x.data.shape}")
print(f"Transformer output shape: {output.data.shape}")

Transformer input shape: (2, 10, 64)
Transformer output shape: (2, 10, 64)


## Spectral Normalization

In [20]:
from fit.nn.utils.spectral_norm import SpectralNormLinear

# Linear layer with spectral normalization
spec_linear = SpectralNormLinear(10, 5, n_power_iterations=1)

x = Tensor(np.random.randn(3, 10), requires_grad=True)
output = spec_linear(x)

print(f"Output shape: {output.data.shape}")

Output shape: (3, 5)


# Monitoring

## Training Tracker

In [21]:
from fit.monitor.tracker import TrainingTracker

# Create tracker
tracker = TrainingTracker(experiment_name="demo_experiment")

# Simulate training with metrics
for epoch in range(10):
    # Simulate epoch metrics
    train_loss = 1.0 - epoch * 0.1 + np.random.normal(0, 0.05)
    val_loss = 1.2 - epoch * 0.08 + np.random.normal(0, 0.08)
    accuracy = epoch * 0.08 + np.random.normal(0, 0.02)

    # Log metrics
    tracker.update(
        epoch, {"train_loss": train_loss, "val_loss": val_loss, "accuracy": accuracy}
    )

    tracker.log_learning_rate(0.001 * (0.9**epoch))

print("Metrics logged!")
print(f"Best validation loss: {tracker.best_values.get('val_loss', 'N/A')}")

# Export metrics
tracker.export("training_log.json")

Epoch train_loss  val_loss    accuracy    Time    
--------------------------------------------------
1     0.9389      0.9989      0.0152      
2     0.8907      1.1735      0.0667      
3     0.8214      1.1364      0.1414      
4     0.7042      0.9471      0.2042      
5     0.5579      0.8620      0.3140      
6     0.5659      0.7661      0.3624      
7     0.3904      0.8223      0.4745      
8     0.3932      0.5757      0.5251      
9     0.3083      0.4118      0.6616      
10    0.1303      0.5031      0.7300      
Metrics logged!
Best validation loss: 0.41183470041323994
Logs exported to training_log.json


'training_log.json'

## Model performance monitoring

In [22]:
from fit.utils.engine import evaluate

# Create a simple model for demonstration
model = Sequential(Linear(4, 8), ReLU(), Linear(8, 3), Softmax())

# Create sample data
X = Tensor(np.random.randn(20, 4))
y = Tensor(np.random.randint(0, 3, 20))
dataset = Dataset(X.data, y.data)
dataloader = DataLoader(dataset, batch_size=5)

# Evaluate model
from fit.loss.classification import CrossEntropyLoss

loss_fn = CrossEntropyLoss()

metrics = evaluate(model, dataloader, loss_fn)
print(f"Evaluation metrics: {metrics}")

Evaluation metrics: {'loss': np.float64(1.0865359454049197), 'accuracy': np.float64(0.5)}


# Model save/load

## Save and load models

In [23]:
from fit.nn.utils.model_io import save_model, load_model

# Create and train a simple model
model = Sequential(Linear(2, 4), ReLU(), Linear(4, 1))

# Save model
save_model(model, "demo_model.pkl")
print("Model saved!")

# Load model
loaded_model = load_model("demo_model.pkl")
print("Model loaded!")

# Test that loaded model works
test_input = Tensor([[1.0, 2.0]])
original_output = model(test_input)
loaded_output = loaded_model(test_input)

print(f"Original output: {original_output.data}")
print(f"Loaded output: {loaded_output.data}")
print(f"Outputs match: {np.allclose(original_output.data, loaded_output.data)}")

Model saved to demo_model.pkl
Model saved!
Model loaded from demo_model.pkl
Model loaded!
Original output: [[-0.21497577]]
Loaded output: [[-0.21497577]]
Outputs match: True


# Examples of usage

## XOR problem

In [24]:
from fit.simple.trainer import SimpleTrainer

# Create XOR model
xor_model = Sequential(Linear(2, 8), ReLU(), Linear(8, 4), ReLU(), Linear(4, 1))

# XOR data
X = Tensor([[0, 0], [0, 1], [1, 0], [1, 1]])
y = Tensor([[0], [1], [1], [0]])

# Create trainer
trainer = SimpleTrainer(
    model=xor_model,
    data=(X, y),
    loss='mse',
    optimizer='adam',
    lr=0.01
)

# Train the model
print("Training XOR model...")
history = trainer.fit(
    epochs=100,          
    verbose=True
)

# Test the trained model
print("\nXOR Results:")
for i, (input_val, expected) in enumerate(zip(X.data, y.data)):
    prediction = xor_model(Tensor([input_val]))
    print(
        f"Input: {input_val}, Expected: {expected[0]:.0f}, Predicted: {float(prediction.data.flatten()[0]):.3f}"
    )

Training XOR model...
Starting training for 100 epochs...
Model: Sequential
Optimizer: Adam
Loss: MSELoss
Batch size: 32
--------------------------------------------------
Epoch train_loss  Time    
--------------------------
1     0.5087      
Epoch 1: train_loss=0.5087
2     0.4190      
Epoch 2: train_loss=0.4190
3     0.3445      
Epoch 3: train_loss=0.3445
4     0.2844      
Epoch 4: train_loss=0.2844
5     0.2368      
Epoch 5: train_loss=0.2368
6     0.1989      
Epoch 6: train_loss=0.1989
7     0.1719      
Epoch 7: train_loss=0.1719
8     0.1514      
Epoch 8: train_loss=0.1514
9     0.1336      
Epoch 9: train_loss=0.1336
10    0.1180      
Epoch 10: train_loss=0.1180
11    0.1040      
Epoch 11: train_loss=0.1040
12    0.0917      
Epoch 12: train_loss=0.0917
13    0.0811      
Epoch 13: train_loss=0.0811
14    0.0721      
Epoch 14: train_loss=0.0721
15    0.0646      
Epoch 15: train_loss=0.0646
16    0.0587      
Epoch 16: train_loss=0.0587
17    0.0538      
Epoch 17: tr

## Multi-class classification

In [25]:
# Load Iris dataset
iris_data = load_dataset("iris", batch_size=16, validation_split=0.3)

# Create classification model
classifier = Sequential(
    Linear(4, 16), ReLU(), Linear(16, 8), ReLU(), Linear(8, 3), Softmax()
)

# Create trainer
trainer = SimpleTrainer(
    model=classifier,
    data=iris_data['train'],
    validation_data=iris_data['val'], 
    loss="crossentropy", 
    optimizer="adam", 
    lr=0.01
)

# Train
print("Training Iris classifier...")
history = trainer.fit(
    epochs=50, 
    verbose=True
)

# Evaluate
final_metrics = evaluate(classifier, iris_data["val"], CrossEntropyLoss())
print(f"Final validation metrics: {final_metrics}")

Loading Iris dataset...
Training Iris classifier...
Starting training for 50 epochs...
Model: Sequential
Optimizer: Adam
Loss: CrossEntropyLoss
Batch size: 16
--------------------------------------------------
Epoch train_loss  val_loss    val_accuracyTime    
--------------------------------------------------
1     0.9388      0.7904      0.7111      
Epoch 1: train_loss=0.9388, val_loss=0.7904, val_acc=0.7111
2     0.7772      0.7367      0.7556      
Epoch 2: train_loss=0.7772, val_loss=0.7367, val_acc=0.7556
3     0.7353      0.7237      0.8222      
Epoch 3: train_loss=0.7353, val_loss=0.7237, val_acc=0.8222
4     0.6990      0.7130      0.8222      
Epoch 4: train_loss=0.6990, val_loss=0.7130, val_acc=0.8222
5     0.6880      0.6977      0.8222      
Epoch 5: train_loss=0.6880, val_loss=0.6977, val_acc=0.8222
6     0.6643      0.6939      0.8444      
Epoch 6: train_loss=0.6643, val_loss=0.6939, val_acc=0.8444
7     0.6346      0.6692      0.8889      
Epoch 7: train_loss=0.6346,

## Custom training loop

In [26]:
from fit.optim.adam import Adam
from fit.loss.regression import MSELoss

# Generate synthetic regression data
np.random.seed(42)
X_train = np.random.randn(100, 3)
true_weights = np.array([1.5, -2.0, 0.5])
y_train = X_train @ true_weights + 0.1 * np.random.randn(100)

# Create model
regression_model = Sequential(Linear(3, 8), ReLU(), Linear(8, 1))

# Setup training
optimizer = Adam(regression_model.parameters(), lr=0.01)
loss_fn = MSELoss()

# Custom training loop
print("Training loop for regression...")
for epoch in range(100):
    # Convert to tensors
    X_tensor = Tensor(X_train, requires_grad=True)
    y_tensor = Tensor(y_train.reshape(-1, 1))

    # Forward pass
    predictions = regression_model(X_tensor)
    loss = loss_fn(predictions, y_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {float(loss.data):.4f}")

print("Training completed!")

# Test prediction
test_X = Tensor([[1.0, -1.0, 0.5]])
prediction = regression_model(test_X)
expected = 1.0 * 1.5 + (-1.0) * (-2.0) + 0.5 * 0.5  # Using true weights
print(f"Test prediction: {float(prediction.data[0]):.3f}, Expected: {expected:.3f}")

Training loop for regression...
Epoch 0: Loss = 13.2949
Epoch 20: Loss = 4.1434
Epoch 40: Loss = 0.9452
Epoch 60: Loss = 0.4565
Epoch 80: Loss = 0.2559
Training completed!
Test prediction: 3.706, Expected: 3.750


/tmp/ipykernel_1677/1613204889.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Test prediction: {float(prediction.data[0]):.3f}, Expected: {expected:.3f}")
